In [ ]:
import pandas as pd
import numpy as np
!pip install geopandas
import geopandas as gpd
import shapely
from shapely.geometry import Point, MultiPolygon
import plotly.express as px

In [56]:
bases_originais = 'bases_originais/'

In [53]:
arquivos_gerados = 'arquivos-gerados/'

# Leitura e formatação de arquivos

In [4]:
# Leitura dos dados de 2015 a 2020:

anos =  ['2015','2016','2017','2018','2019','2020']
sinistros = []
for ano in anos:
  df = pd.read_csv(f'{bases_originais}/2015_2020/Sinistros_{ano}.csv',sep=';', encoding='iso-8859-1')
  # print(f'{ano}ok')
  df['ano'] = ano
  sinistros.append(df)
sinistros = pd.concat(sinistros)

# Leitura e tratamento dos dados de 2013, 2014, 2021 e 2022 / Nova Versão: 

anos = ['2021', '2022']

# 2022 coletado em 

concat_dataset = []

for a in anos:

  veiculos = pd.read_csv(f'{bases_originais}{a}/veiculos.csv',sep=';', encoding='iso-8859-1', index_col=False)
  incidents = pd.read_csv(f'{bases_originais}{a}/incidents.csv',sep=';', encoding='iso-8859-1', index_col=False)
  vitimas = pd.read_csv(f'{bases_originais}{a}/vitimas.csv',sep=';', encoding='iso-8859-1', index_col=False)

  dataset_ano = pd.merge(pd.merge(incidents, veiculos, left_index=True, right_index=True), vitimas, left_index=True, right_index=True) 
  dataset_ano['ano'] = a

  # LATITUDE E LONGITUDE POR COORDENADAS: 
  aux = dataset_ano['COORDENADAS'].str.strip('()').str.split(',').str[:2]
  dataset_ano['LATITUDE'] = aux.apply(lambda x: x[0].split(' ')[0])
  dataset_ano['LONGITUDE'] = aux.apply(lambda x: x[0].split(' ')[1])

  # ANO, DIA, MES, HORA a partir da DATA
  dataset_ano['DATA'] = pd.to_datetime(dataset_ano['DATA'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
  dataset_ano['ANO'] = dataset_ano['DATA'].dt.year
  dataset_ano['MES'] = dataset_ano['DATA'].dt.month
  dataset_ano['DIA'] = dataset_ano['DATA'].dt.day
  dataset_ano['HORA'] = dataset_ano['DATA'].dt.hour

  # Removendo colunas: 
  dataset_ano = dataset_ano.drop(['COD_ACIDENTE_x', 'COD_ACIDENTE_y','DATA'], axis=1)

  dataset_ano.loc[dataset_ano['LATITUDE'] == '0','LATITUDE'] = np.nan

  concat_dataset.append(dataset_ano)

concat_dataset = pd.concat(concat_dataset)

sinistros = pd.concat([sinistros, concat_dataset])

In [5]:
lista_colunas = ['COD_ACIDENTE', 'INTERSECAO', 'NUMERO', 'DIA', 'MES', 'ANO']
for coluna in lista_colunas:
  sinistros[coluna] = sinistros[coluna].astype(str)

In [6]:
# Formatar data
sinistros['dia/mes/ano'] =  sinistros.apply(lambda x: f'{x["DIA"]}/{x["MES"]}/{x["ANO"]}', axis=1)
sinistros['dia/mes/ano'] = pd.to_datetime(sinistros['dia/mes/ano'], format='%d/%m/%Y')

sinistros['DATA'] = sinistros['dia/mes/ano'].astype(str)
sinistros['ANO/MES'] = sinistros['DATA'].str.slice(start=0, stop=7)

In [7]:
sinistros.to_csv(arquivos_gerados+'dataset_sinistros.csv')

# Análises

## Descritiva

In [8]:
sinistros.LATITUDE.isna().value_counts()

False    97294
True     26463
Name: LATITUDE, dtype: int64

In [9]:
# sinistros.loc[sinistros['LATITUDE'].notnull()].to_csv(arquivos_gerados+'dataset_sinistros_com_localizacao.csv')

In [10]:
# Gráfico de sinistros totais por ano

sinistros_ano_total = sinistros[['COD_ACIDENTE', 'ANO']]
sinistros_ano_total_groupby = sinistros_ano_total.groupby('ANO').count().reset_index()

fig1 = px.bar(sinistros_ano_total_groupby, x="ANO", y="COD_ACIDENTE", text="COD_ACIDENTE", title='Total de sinistros de trânsito por ano em Fortaleza')
fig1.update_traces(textposition="outside")
fig1.show()

In [11]:
sinistros_ano_total_groupby.rename(columns={'COD_ACIDENTE': 'TOTAL DE SINISTROS'}, inplace=True)

In [12]:
sinistros_ano_total_groupby.to_csv(arquivos_gerados+'sinistros_por_ano.csv', index=False)

In [13]:
# Gráfico de sinistros com morte por ano

sinistros_mortos = sinistros[['COD_ACIDENTE', 'ANO', 'MORTOS']]
sinistros_mortos['MORTOS'] = sinistros_mortos['MORTOS'].astype(int)
sinistros_mortos_total = sinistros_mortos[sinistros_mortos['MORTOS'] != 0]
sinistros_mortos_total_groupby = sinistros_mortos_total.groupby('ANO').sum().reset_index()

fig2 = px.bar(sinistros_mortos_total_groupby, x="ANO", y="MORTOS", text="MORTOS", title='Total de mortos em sinistros de trânsito por ano em Fortaleza')
fig2.update_traces(textposition="outside")
fig2.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Sinistros com bicicleta

In [14]:
sinistros_bicicletas = sinistros[['COD_ACIDENTE', 'MORTOS', 'FERIDOS', 'ILESOS', 'AUTOMOVEL', 'ONIBUS', 'BICICLETA', 'CAMINHAO', 'MICROONIBUS', 'CICLOMOTOR', 'MOTOCICLETA', 'TRACAOANIMAL', 'TREM', 'MES', 'ANO', 'ANO/MES']][sinistros.BICICLETA == 1]
sinistros_bicicletas.tail()

,COD_ACIDENTE,MORTOS,FERIDOS,ILESOS,AUTOMOVEL,ONIBUS,BICICLETA,CAMINHAO,MICROONIBUS,CICLOMOTOR,MOTOCICLETA,TRACAOANIMAL,TREM,MES,ANO,ANO/MES
992,BVK2540,0,1,0,1,0,1,0,0,0,0,0,0,1,2022,2022-01
1025,FPG8738,0,1,0,1,0,1,0,0,0,0,0,0,1,2022,2022-01
1040,OKK8827,0,0,1,1,0,1,0,0,0,0,0,0,1,2022,2022-01
1053,RRJ0049,0,0,0,1,0,1,0,0,0,0,0,0,1,2022,2022-01
1095,OZG4505,1,1,0,0,0,1,0,0,0,1,0,0,1,2022,2022-01


In [15]:
sinistros_bicicletas[['COD_ACIDENTE', 'ANO']].groupby('ANO').count().reset_index().to_csv(arquivos_gerados+'dataset_sinistros_com_bicicleta_por_mes.csv')

In [16]:
df1_sinistros_bicicletas = sinistros[['COD_ACIDENTE', 'ANO/MES', 'MORTOS', 'FERIDOS', 'ILESOS']][sinistros.BICICLETA == 1]
df2_sinistros_bicicletas = sinistros[['COD_ACIDENTE', 'ANO/MES', 'AUTOMOVEL', 'ONIBUS', 'BICICLETA', 'CAMINHAO', 'MICROONIBUS', 'CICLOMOTOR', 'MOTOCICLETA', 'TRACAOANIMAL', 'TREM']][sinistros.BICICLETA == 1]

In [17]:
sinistros_vitimas = df1_sinistros_bicicletas.melt(id_vars=['COD_ACIDENTE', 'ANO/MES']).rename(columns={'variable': 'VITIMAS', 'value': 'QUANTIDADE'})
sinistros_vitimas.head()

,COD_ACIDENTE,ANO/MES,VITIMAS,QUANTIDADE
0,256302,2015-01,MORTOS,0
1,247631,2015-01,MORTOS,0
2,256469,2015-01,MORTOS,0
3,256553,2015-01,MORTOS,0
4,256492,2015-01,MORTOS,0


In [18]:
sinistros_veiculos = df2_sinistros_bicicletas.melt(id_vars=['COD_ACIDENTE', 'ANO/MES']).rename(columns={'variable': 'VEICULOS ENVOLVIDOS', 'value': 'QUANTIDADE'})
sinistros_veiculos.head()

,COD_ACIDENTE,ANO/MES,VEICULOS ENVOLVIDOS,QUANTIDADE
0,256302,2015-01,AUTOMOVEL,0
1,247631,2015-01,AUTOMOVEL,1
2,256469,2015-01,AUTOMOVEL,1
3,256553,2015-01,AUTOMOVEL,1
4,256492,2015-01,AUTOMOVEL,0


In [19]:
df_sinistros_veiculos = sinistros_veiculos.groupby(['ANO/MES', 'VEICULOS ENVOLVIDOS']).sum().reset_index()
df_sinistros_veiculos['VEICULOS ENVOLVIDOS'] = sinistros_veiculos['VEICULOS ENVOLVIDOS'].str.replace('AUTOMOVEL', 'Automóvel').str.replace('MOTOCICLETA', 'Motocicleta').str.replace('CICLOMOTOR', 'Ciclomotor').str.replace('TREM', 'Trem').str.replace('TRACAOANIMAL', 'Tração animal').str.replace('BICICLETA', 'Bicicleta').str.replace('MICROONIBUS', 'Microônibus').str.replace('ONIBUS', 'Ônibus').str.replace('CAMINHAO', 'Caminhão')
df_sinistros_veiculos = sinistros_veiculos[sinistros_veiculos['QUANTIDADE'] != 0]
df_sinistros_veiculos.rename(columns={'ANO/MES': 'Ano/mês', 'VEICULOS ENVOLVIDOS': 'Veículos envolvidos', 'QUANTIDADE': 'Quantidade'}, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
# dataframe com percentual, por ano e mês, de sinistros que envolvem bicicleta ou não

sinistros_ano_mes = sinistros[['ANO/MES', 'BICICLETA']].value_counts().reset_index().rename(columns={0:'QUANTIDADE'})
sinistros_ano_mes['BICICLETA'] = sinistros_ano_mes['BICICLETA'].astype(str).str.replace('0', 'Não envolve bicicleta').str.replace('1', 'Envolve bicicleta')
sinistros_ano_mes_pvt = pd.pivot_table(sinistros_ano_mes, columns='BICICLETA', index='ANO/MES').reset_index()
sinistros_ano_mes_pvt['TOTAL'] = sinistros_ano_mes_pvt.sum(axis=1)
sinistros_ano_mes_pvt[('PERCENTUAL', '% COM BICICLETAS')] = round(sinistros_ano_mes_pvt[('QUANTIDADE',     'Envolve bicicleta')] / sinistros_ano_mes_pvt['TOTAL']*100,2)
sinistros_ano_mes_pvt[('PERCENTUAL', '% SEM BICICLETAS')] = round(sinistros_ano_mes_pvt[('QUANTIDADE', 'Não envolve bicicleta')] / sinistros_ano_mes_pvt['TOTAL']*100,2)
sinistros_ano_mes_pvt = sinistros_ano_mes_pvt.fillna(0)
sinistros_ano_mes_pvt.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



ANO/MES        QUANTIDADE                         TOTAL  \
BICICLETA          Envolve bicicleta Não envolve bicicleta           
0          2015-01              38.0                2252.0  2290.0   
1          2015-02              22.0                1758.0  1780.0   
2          2015-03              20.0                1921.0  1941.0   
3          2015-04              25.0                1722.0  1747.0   
4          2015-05              28.0                2135.0  2163.0   

                PERCENTUAL                   
BICICLETA % COM BICICLETAS % SEM BICICLETAS  
0                     1.66            98.34  
1                     1.24            98.76  
2                     1.03            98.97  
3                     1.43            98.57  
4                     1.29            98.71

In [21]:
# dataframe com percentual, por ano, de sinistros que envolvem bicicleta ou não

sinistros_ano = sinistros[['ANO', 'BICICLETA']].value_counts().reset_index().rename(columns={0:'QUANTIDADE'})
sinistros_ano['BICICLETA'] = sinistros_ano['BICICLETA'].astype(str).str.replace('0', 'Não envolve bicicleta').str.replace('1', 'Envolve bicicleta')
sinistros_ano_pvt = pd.pivot_table(sinistros_ano, columns='BICICLETA', index='ANO').reset_index()
sinistros_ano_pvt['TOTAL'] = sinistros_ano_pvt.sum(axis=1)
sinistros_ano_pvt[('PERCENTUAL', '% COM BICICLETAS')] = round(sinistros_ano_pvt[('QUANTIDADE',     'Envolve bicicleta')] / sinistros_ano_pvt['TOTAL']*100,2)
sinistros_ano_pvt[('PERCENTUAL', '% SEM BICICLETAS')] = round(sinistros_ano_pvt[('QUANTIDADE', 'Não envolve bicicleta')] / sinistros_ano_pvt['TOTAL']*100,2)
sinistros_ano_pvt.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



ANO        QUANTIDADE                        TOTAL  \
BICICLETA       Envolve bicicleta Não envolve bicicleta          
0          2015               347                 23211  23558   
1          2016               721                 26768  27489   
2          2017               728                 19572  20300   
3          2018               760                 13934  14694   
4          2019               881                 13644  14525   

                PERCENTUAL                   
BICICLETA % COM BICICLETAS % SEM BICICLETAS  
0                     1.47            98.53  
1                     2.62            97.38  
2                     3.59            96.41  
3                     5.17            94.83  
4                     6.07            93.93

In [22]:
sinistros_ano_pvt.to_csv(arquivos_gerados+'sinistros_ano_com_e_sem_bicicleta.csv', index=False)

In [23]:
# Sinistros com e sem bicicleta: números absolutos e percentual 

envolve_bicicleta = pd.DataFrame(sinistros.BICICLETA.value_counts()).reset_index()
envolve_bicicleta['PERCENTUAL'] = round(envolve_bicicleta['BICICLETA']/envolve_bicicleta['BICICLETA'].sum()*100,2)
envolve_bicicleta = envolve_bicicleta.rename(columns={'index': 'ENVOLVE BICICLETA', 'BICICLETA': 'QUANTIDADE'})
envolve_bicicleta['ENVOLVE BICICLETA'][0] = 'NÃO'
envolve_bicicleta['ENVOLVE BICICLETA'][1] = 'SIM'
envolve_bicicleta

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,ENVOLVE BICICLETA,QUANTIDADE,PERCENTUAL
0,NÃO,118920,96.09
1,SIM,4837,3.91


In [24]:
sinistros_bicicleta = sinistros.loc[sinistros.BICICLETA == 1]
sinistros_com_bicicleta_por_ano = pd.DataFrame(sinistros_bicicleta['ANO'].value_counts()).reset_index()
sinistros_com_bicicleta_por_ano = sinistros_com_bicicleta_por_ano.rename(columns={'index':'ANO', 'ANO': 'SINISTROS COM BICICLETAS'}).sort_values(by='ANO')
sinistros_com_bicicleta_por_ano

,ANO,SINISTROS COM BICICLETAS
6,2015,347
3,2016,721
2,2017,728
1,2018,760
0,2019,881
5,2020,674
4,2021,696
7,2022,30


In [25]:
sinistros_com_bicicleta_por_ano.to_csv(arquivos_gerados+'sinistros_com_bicicleta_por_ano.csv', index=False)

In [26]:
fig1 = px.bar(sinistros_com_bicicleta_por_ano, x="ANO", y="SINISTROS COM BICICLETAS", text="SINISTROS COM BICICLETAS", title='Quantidade de sinistros envolvendo bicicleta por ano em Fortaleza')
fig1.update_traces(textposition="outside")
fig1.show()

In [27]:
sinistros_bicicleta_iluminacao = sinistros_bicicleta[['ANO', 'ILUMINACAO']].value_counts().reset_index().rename(columns={'index': 'Iluminação', 'ILUMINACAO': 'Qtd sinistros com bicicletas'})
sinistros_bicicleta_iluminacao_pvt = pd.pivot_table(sinistros_bicicleta_iluminacao, columns='ANO', index='Qtd sinistros com bicicletas').reset_index().fillna(0)
sinistros_bicicleta_iluminacao_pvt

Qtd sinistros com bicicletas      0                                     \
ANO                                2015   2016   2017   2018   2019   2020   
0                      Amanhecer    3.0   17.0   12.0    7.0   10.0    6.0   
1                      Anoitecer   34.0   86.0   99.0  107.0  132.0  100.0   
2                     Luz do Dia  184.0  382.0  385.0  372.0  429.0  340.0   
3                  Mal Iluminada    1.0    2.0    0.0    5.0    2.0    2.0   
4                 N?úo Informado    0.0    1.0    1.0    0.0    0.0    0.0   
5                 NÃ£o informado    0.0    0.0    0.0    0.0    0.0    0.0   
6                  Via Iluminada   33.0   20.0   27.0   25.0   16.0   14.0   

                  
ANO   2021  2022  
0      0.0   0.0  
1      4.0   2.0  
2     31.0  14.0  
3      2.0   2.0  
4      0.0   0.0  
5    637.0   2.0  
6     20.0  10.0

## Dia da semana e hora

In [28]:
sinistros['tem_latitude'] = 'Não'
sinistros.loc[sinistros.LATITUDE.notnull(),'tem_latitude'] = "Sim"
sinistros.groupby(['ano','tem_latitude']).size().reset_index().pivot(index='ano',columns='tem_latitude', values=0).fillna(0).astype(int)

tem_latitude,Não,Sim
ano,,
2015,6731,16827
2016,8651,18838
2017,3513,16787
2018,2757,11937
2019,2678,11847
2020,2050,9240
2021,83,10722
2022,0,1096


In [29]:
sinistros['tem_latitude'].value_counts(normalize=True)

Sim    0.78617
Não    0.21383
Name: tem_latitude, dtype: float64

In [30]:
# Dia_hora da semana:
sinistros['Envolve_Bicicleta'] = 'Não'
sinistros.loc[sinistros.BICICLETA!=0, 'Envolve_Bicicleta'] = 'Sim'

In [31]:
sinistros.HORA = sinistros.HORA.apply(lambda x: float(str(x).split(':')[0]))
sinistros['DATA_2'] = pd.to_datetime(sinistros.DATA, format='%Y-%m-%d', errors='coerce')

In [32]:
ref_dia = {0:'Segunda',1:'Terça',2:'Quarta',3:'Quinta',4:'Sexta',5:'Sábado',6:'Domingo'}
sinistros['DIA_DA_SEMANA'] = sinistros.DATA_2.dt.dayofweek

In [33]:
# sinistros[['DATA','DIA_DA_SEMANA']]
resumo_dia_hora = sinistros.groupby(['DIA_DA_SEMANA','HORA']).size().reset_index()
resumo_dia_hora.columns = ['Dia','Hora','Qtd']
total = resumo_dia_hora['Qtd'].sum()
resumo_dia_hora['Percentual'] = (resumo_dia_hora['Qtd']/total)*100

resumo_dia_hora['dia_nome'] = resumo_dia_hora['Dia'].map(ref_dia)
resumo_dia_hora = resumo_dia_hora.sort_values(['Dia','Hora'])
resumo_dia_hora.to_csv('resumo_dia_hora.csv',index=False)
resumo_dia_hora

,Dia,Hora,Qtd,Percentual,dia_nome
0,0,0.0,430,0.348268,Segunda
1,0,1.0,362,0.293193,Segunda
2,0,2.0,237,0.191953,Segunda
3,0,3.0,177,0.143357,Segunda
4,0,4.0,145,0.117439,Segunda
...,...,...,...,...,...
163,6,19.0,1018,0.824505,Domingo
164,6,20.0,1001,0.810736,Domingo
165,6,21.0,915,0.741083,Domingo
166,6,22.0,660,0.534551,Domingo


In [34]:
# sinistros[['DATA','DIA_DA_SEMANA']]
resumo_dia_hora = sinistros.groupby(['DIA_DA_SEMANA','HORA','Envolve_Bicicleta']).size().reset_index()
resumo_dia_hora.columns = ['Dia','Hora','Envolve_Bicicleta','Qtd']
resumo_dia_hora.Hora = resumo_dia_hora.Hora.astype(int)
resumo_dia_hora['dia_nome'] = resumo_dia_hora['Dia'].map(ref_dia)
resumo_dia_hora = resumo_dia_hora.sort_values(['Dia','Hora'])
resumo_dia_hora.to_csv('resumo_dia_hora_envolvimento_bike.csv',index=False)
resumo_dia_hora

,Dia,Hora,Envolve_Bicicleta,Qtd,dia_nome
0,0,0,Não,416,Segunda
1,0,0,Sim,14,Segunda
2,0,1,Não,352,Segunda
3,0,1,Sim,10,Segunda
4,0,2,Não,233,Segunda
...,...,...,...,...,...
331,6,21,Sim,31,Domingo
332,6,22,Não,629,Domingo
333,6,22,Sim,31,Domingo
334,6,23,Não,443,Domingo


## Geolocalização

In [58]:
bairros_fortaleza = gpd.read_file(bases_originais +'dataset_bairros.geojson')

In [36]:
bairros_fortaleza.columns

Index(['id', 'bairro', 'idhm_2010', 'ext_eixos_viarios',
       'ext_eixo_cicloviario_atual', 'prop_cic_total', 'Faixa IDH',
       'NOME_BAIRRO', 'geometry'],
      dtype='object')

In [37]:
def getBairro(Point, coluna_desejada_gBairros):
    query = bairros_fortaleza[bairros_fortaleza['geometry'].contains(Point)]
    if query.shape[0] > 0:
        return query.iloc[0][coluna_desejada_gBairros]
    else:
        return 'Não identificado'

In [38]:
sinistros_envolvendo_bicicletas = sinistros.loc[sinistros.BICICLETA != 0 ]

In [39]:
sinistros_bicicleta_com_localizacao = sinistros_envolvendo_bicicletas.loc[~sinistros_envolvendo_bicicletas.LATITUDE.isna()]
sinistros_bicicleta_com_localizacao.head(3)

,COD_ACIDENTE,DIA,MES,ANO,HORA,INTERSECAO,LOG1,LOG2,LOG3,NUMERO,...,AMC,COORDENADAS,OUTROS,dia/mes/ano,DATA,ANO/MES,tem_latitude,Envolve_Bicicleta,DATA_2,DIA_DA_SEMANA
5,256302,1,1,2015,2.0,3778.0,RUA ANTONIO POMPEU,RUA GAL SAMPAIO,NaN,0.0,...,NaN,NaN,NaN,2015-01-01,2015-01-01,2015-01,Sim,Sim,2015-01-01,3
84,247631,2,1,2015,15.0,5295.0,AV IMPERADOR,RUA PEDRO I,NaN,0.0,...,NaN,NaN,NaN,2015-01-02,2015-01-02,2015-01,Sim,Sim,2015-01-02,4
136,256469,2,1,2015,18.0,5295.0,RUA PEDRO I,AV IMPERADOR,NaN,0.0,...,NaN,NaN,NaN,2015-01-02,2015-01-02,2015-01,Sim,Sim,2015-01-02,4


In [ ]:
sinistros_bicicleta_com_localizacao['lat'] = sinistros_bicicleta_com_localizacao.apply(lambda x: float(x.LATITUDE.replace(',','.').replace(';','.')), axis=1)
sinistros_bicicleta_com_localizacao['lng'] = sinistros_bicicleta_com_localizacao.apply(lambda x: float(x.LONGITUDE.replace(',','.').replace(';','.')), axis=1)

sinistros_bicicleta_com_localizacao['Point'] = sinistros_bicicleta_com_localizacao.apply(lambda x: Point(x['lng'],x['lat']), axis=1)

In [ ]:
sinistros_bicicleta_com_localizacao['bairro'] = sinistros_bicicleta_com_localizacao['Point'].apply(lambda x: getBairro(x,'bairro'))
sinistros_bicicleta_com_localizacao['faixa_idh'] = sinistros_bicicleta_com_localizacao['Point'].apply(lambda x: getBairro(x,'Faixa IDH'))

In [42]:
sinistros_bicicleta_com_localizacao.groupby(['ano','faixa_idh']).size().reset_index().pivot(index='ano',columns='faixa_idh',values=0).fillna(0).astype(int)

faixa_idh,Alto,Baixo,Muito Alto,Muito Baixo,Médio,Não identificado
ano,,,,,,
2015,5,40,22,157,13,2
2016,13,65,27,318,29,0
2017,14,57,36,375,31,3
2018,13,62,42,416,35,5
2019,15,86,45,515,42,2
2020,13,61,42,383,27,2
2021,0,0,0,0,0,694
2022,0,0,0,0,0,30


In [43]:
sinistros_bicicleta_com_localizacao.groupby(['faixa_idh']).size().reset_index().rename(columns={'faixa_idh': 'Faixa de IDH', 0: 'Quantidade'})

,Faixa de IDH,Quantidade
0,Alto,73
1,Baixo,371
2,Muito Alto,214
3,Muito Baixo,2164
4,Médio,177
5,Não identificado,738


In [44]:
df_sinistros_bairros = pd.DataFrame(sinistros_bicicleta_com_localizacao['bairro'].value_counts()).reset_index().rename(columns={'index': 'bairro', 'bairro': 'quantidade'})
df_sinistros_bairros.head()

,bairro,quantidade
0,Não identificado,738
1,Centro,135
2,Meireles,92
3,Aldeota,90
4,Messejana,88


In [ ]:
# geolocalização com todos os sinistros: 

sinistros_com_localizacao = sinistros.loc[~sinistros.LATITUDE.isna()]

sinistros_com_localizacao['lat'] = sinistros_com_localizacao.apply(lambda x: float(x.LATITUDE.replace('.','').replace(',','.').replace(';','.')), axis=1)
sinistros_com_localizacao['lng'] = sinistros_com_localizacao.apply(lambda x: float(x.LONGITUDE.replace('.','').replace(',','.').replace(';','.')), axis=1)

sinistros_com_localizacao['Point'] = sinistros_com_localizacao.apply(lambda x: Point(x['lng'],x['lat']), axis=1)

In [ ]:
sinistros_com_localizacao['bairro'] = sinistros_com_localizacao['Point'].apply(lambda x: getBairro(x,'bairro'))
sinistros_com_localizacao['faixa_idh'] = sinistros_com_localizacao['Point'].apply(lambda x: getBairro(x,'Faixa IDH'))

In [47]:
# Resumo dos sinistros por faixa de IDH dos bairros, envolvendo ou não bicicleta, em números absolutos e percentual

dataset_resumo = sinistros_com_localizacao.groupby(['faixa_idh','Envolve_Bicicleta']).size().reset_index().pivot(index='Envolve_Bicicleta',columns='faixa_idh',values=0).fillna(0).astype(int)
dataset_resumo = sinistros_com_localizacao.groupby(['faixa_idh','Envolve_Bicicleta']).size().reset_index()
dataset_resumo.columns = ['faixa_idh','Envolve_Bicicleta','Qtd']
dataset_resumo = dataset_resumo.pivot(index='faixa_idh', columns ='Envolve_Bicicleta', values='Qtd').reset_index()
dataset_resumo['pct'] = round(dataset_resumo['Sim']/dataset_resumo['Não']*100,2)
dataset_resumo

Envolve_Bicicleta,faixa_idh,Não,Sim,pct
0,Alto,2525,73,2.89
1,Baixo,13203,371,2.81
2,Muito Alto,8219,214,2.60
3,Muito Baixo,51174,2164,4.23
4,Médio,6960,177,2.54
5,Não identificado,11476,738,6.43


In [48]:
# Dos que envolve bicicleta, quantos eram fatais e qual a proporção por faixa de IDH

resumo_obitos = sinistros_com_localizacao.loc[sinistros_com_localizacao.BICICLETA != 0]

resumo_obitos['Mortos?'] = 'Não'
resumo_obitos.loc[resumo_obitos.MORTOS != 0, 'Mortos?'] = "Sim"

resumo_obitos = resumo_obitos.groupby(['faixa_idh','Mortos?']).size().reset_index()
resumo_obitos.columns = ['faixa_idh','Mortos?','Qtd']

resumo_obitos=resumo_obitos.pivot(index='faixa_idh', columns ='Mortos?', values='Qtd').reset_index().fillna(0)


resumo_obitos['pct'] = round(resumo_obitos['Sim']/resumo_obitos['Não']*100,2)

resumo_obitos

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Mortos?,faixa_idh,Não,Sim,pct
0,Alto,73.0,0.0,0.00
1,Baixo,360.0,11.0,3.06
2,Muito Alto,210.0,4.0,1.90
3,Muito Baixo,2066.0,98.0,4.74
4,Médio,173.0,4.0,2.31
5,Não identificado,719.0,19.0,2.64


In [49]:
# Do resumo de sinistros com óbitos, vamos localizar os bairros onde ocorreram

resumo_obitos = sinistros_com_localizacao.loc[sinistros_com_localizacao.BICICLETA != 0]

resumo_obitos['Mortos?'] = 'Não'
resumo_obitos.loc[resumo_obitos.MORTOS != 0, 'Mortos?'] = "Sim"

resumo_obitos = resumo_obitos.groupby(['bairro','Mortos?']).size().reset_index()
resumo_obitos.columns = ['bairro','Mortos?','Qtd']

resumo_obitos=resumo_obitos.pivot(index='bairro', columns ='Mortos?', values='Qtd').reset_index().fillna(0)

resumo_obitos['Sim'] = resumo_obitos['Sim'].astype(int)
resumo_obitos['Não'] = resumo_obitos['Não'].astype(int)

resumo_obitos['pct'] = round(resumo_obitos['Sim']/resumo_obitos['Não']*100,2)

resumo_obitos

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Mortos?,bairro,Não,Sim,pct
0,Aerolândia,13,1,7.69
1,Aeroporto,18,1,5.56
2,Aldeota,88,2,2.27
3,Alto da Balança,18,0,0.00
4,Amadeu Furtado,33,1,3.03
...,...,...,...,...
116,Vicente Pinzón,32,1,3.12
117,Vila Peri,24,1,4.17
118,Vila União,22,1,4.55
119,Vila Velha,36,2,5.56


In [50]:
merge_dataset_bairros = pd.merge(bairros_fortaleza[['bairro','prop_cic_total','Faixa IDH']],resumo_obitos, on='bairro')[['bairro','prop_cic_total','pct','Faixa IDH','Não','Sim']]
merge_dataset_bairros

,bairro,prop_cic_total,pct,Faixa IDH,Não,Sim
0,Parque Araxá,29.0,0.00,Baixo,6,0
1,José Bonifácio,27.0,0.00,Médio,35,0
2,Parquelândia,25.0,4.55,Médio,22,1
3,Moura Brasil,23.0,11.11,Muito Baixo,9,1
4,Varjota,22.0,0.00,Alto,11,0
...,...,...,...,...,...,...
115,Conjunto Esperança,0.0,0.00,Muito Baixo,12,0
116,Ellery,0.0,0.00,Muito Baixo,8,0
117,Floresta,0.0,0.00,Muito Baixo,18,0
118,Manuel Dias Branco,0.0,25.00,Muito Baixo,4,1
